In [1]:
%%time
import json
import pandas as pd
import numpy as np
import cv2
from PIL import Image
#from tesserocr import PyTessBaseAPI, PSM, RIL, PT
import matplotlib.pyplot as plt

CPU times: user 599 ms, sys: 440 ms, total: 1.04 s
Wall time: 1min 28s


In [2]:
%%time
import lib.utils as utils
import lib.config as config
import lib.model as model
import lib.roi as roi
import lib.split_pages as spages
import lib.utils as utils

/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 1.74 s, sys: 1.13 s, total: 2.87 s
Wall time: 2min 58s


In [3]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_1.jpg"
image_dir = "images/cropped/"

In [4]:
images = utils.load_images(image_dir)[:2]

In [5]:
newprompt = """
Process this image in two steps:

Step 1: Turn the text in this picture into markdown. Indicate italics. Indent lines which discuss folder contents as unordered lists

Step 2: Convert the markdown text that you created in step 1 into JSON. Use the heading texts as keys, and the folder details as nested numbered lists

Your output should consist of the markdown text
"""

In [6]:
qwen_model = model.QWEN_Model(prompt=newprompt, batch_size=2)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.53it/s]


In [7]:
outputs = qwen_model.batch_infer(images, save=False, debug=True)

>>> Using: 
 	Maximum new tokens = 5000 
 	Batch size = 2 
 	save_path = None
>>> Batch 1 starting...
	Processing text prompts...
	Reading Images...
	Processing inputs...
	Moving inputs to gpu...
	Performing inference...
	Inference Finished
tensor([[151643, 151643, 151643,  ..., 151643, 151643, 151643],
        [151644,   8948,    198,  ...,    935,  73594, 151645]])
	Seperating Ids...
[tensor([ 14374,   3757,   8658,  ..., 151643, 151643, 151643]), tensor([ 73594,  60073,    198,  ...,    935,  73594, 151645])]
	Decoding Ids...
['### John Lightfoot\n\n#### pauciflorus Lightfoot]\n- "Scirpus pauciflorus. Scotland. β" [JL]\n- "My Scirpus pauciflorus. Flor: Scot: appendix. Some of the Specimens by Teesdale [possibly Mr. Teesdale, rather than the place] from Yorkshire. Scheuch. Agr: t. 7, f. 9, Pag: 364. I have examined & found it not to be the Schoenus ferrugineus Lin: because it has no Involucrum mucronatum, & is much smaller & more pliant. It certainly is different from Scirpus caespit

In [8]:
out = outputs[0][1]

In [9]:
out

'### John Lightfoot\n\n#### pauciflorus Lightfoot]\n- "Scirpus pauciflorus. Scotland. β" [JL]\n- "My Scirpus pauciflorus. Flor: Scot: appendix. Some of the Specimens by Teesdale [possibly Mr. Teesdale, rather than the place] from Yorkshire. Scheuch. Agr: t. 7, f. 9, Pag: 364. I have examined & found it not to be the Schoenus ferrugineus Lin: because it has no Involucrum mucronatum, & is much smaller & more pliant. It certainly is different from Scirpus caespitosus as having truncated vaginae & c." [JL]\n\n#### Eleogiton fluitans (L.) Link\n- 2 folders.\n  - Folder 1. Scirpus fluitans [TA], Flora Scotica ed. 1, 88 (1777).\n    - i. Cites Linn. Sp. Pl. 71; Ray\'s Syn. 431; Moris. Sect. 8, t. 10, f. 31; Pluk. t. 35, f. 1.\n    - ii. "Scirpus fluitans" [JL]\n    - iii. "Floating Club-rush. In Gravel Pits that have been full of Water in the Winter, upon Hounslow-Heath, Wandsworth Common, Hillingdon Heath near Uxbridge, & c. July" [JL]\n    - iv. "Hounslow Heath" [?]\n  - Folder 2. Scirpus f